##### Copyright 2021 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 将 LoggingTensorHook 和 StopAtStepHook 迁移到 Keras 回调

<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://tensorflow.google.cn/guide/migrate/logging_stop_hook"><img src="https://tensorflow.google.cn/images/tf_logo_32px.png">在 TensorFlow.org 上查看</a>   </td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/zh-cn/guide/migrate/logging_stop_hook.ipynb"><img src="https://tensorflow.google.cn/images/colab_logo_32px.png">在 Google Colab 运行</a>
</td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/zh-cn/guide/migrate/logging_stop_hook.ipynb"><img src="https://tensorflow.google.cn/images/GitHub-Mark-32px.png">在 Github 上查看源代码</a>
</td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/zh-cn/guide/migrate/logging_stop_hook.ipynb"><img src="https://tensorflow.google.cn/images/download_logo_32px.png">下载笔记本</a>   </td>
</table>

在 TensorFlow 1 中，可以使用 `tf.estimator.LoggingTensorHook` 监视和记录张量，而 `tf.estimator.StopAtStepHook` 则在使用 `tf.estimator.Estimator` 进行训练时有助于在指定步骤停止训练。本笔记本演示了如何使用带有 `Model.fit` 的自定义 Keras 回调 (`tf.keras.callbacks.Callback`) 从这些 API 迁移到 TensorFlow 2 中的对应项。

Keras [回调](https://tensorflow.google.cn/guide/keras/custom_callback)是在内置 Keras `Model.fit`/`Model.evaluate`/`Model.predict` API 中的训练/评估/预测期间的不同点调用的对象。可以在 `tf.keras.callbacks.Callback` API 文档以及[编写自己的回调](../..guide/keras/custom_callback.ipynb/)和[使用内置方法进行训练和评估](https://tensorflow.google.cn/guide/keras/train_and_evaluate)（*使用回调* 部分）指南中详细了解回调。要从 TensorFlow 1 中的 `SessionRunHook` 迁移到 TensorFlow 2 中的 Keras 回调，请查看[迁移使用辅助逻辑的训练](sessionrunhook_callback.ipynb)指南。

## 安装

从导入和用于演示目的的简单数据集开始：

In [ ]:
import tensorflow as tf
import tensorflow.compat.v1 as tf1

In [ ]:
features = [[1., 1.5], [2., 2.5], [3., 3.5]]
labels = [[0.3], [0.5], [0.7]]

# Define an input function.
def _input_fn():
  return tf1.data.Dataset.from_tensor_slices((features, labels)).batch(1)

## TensorFlow 1：使用 tf.estimator API 记录张量和停止训练

在 TensorFlow 1 中，定义各种钩子来控制训练行为。随后，将这些钩子传递给 `tf.estimator.EstimatorSpec`。

在下面的示例中：

- 要监视/记录张量（例如模型权重或损失），可以使用 `tf.estimator.LoggingTensorHook`（`tf.train.LoggingTensorHook` 是它的别名）。
- 要在特定步骤停止训练，请使用 `tf.estimator.StopAtStepHook`（`tf.train.StopAtStepHook` 是它的别名）。

In [ ]:
def _model_fn(features, labels, mode):
  dense = tf1.layers.Dense(1)
  logits = dense(features)
  loss = tf1.losses.mean_squared_error(labels=labels, predictions=logits)
  optimizer = tf1.train.AdagradOptimizer(0.05)
  train_op = optimizer.minimize(loss, global_step=tf1.train.get_global_step())

  # Define the stop hook.
  stop_hook = tf1.train.StopAtStepHook(num_steps=2)

  # Access tensors to be logged by names.
  kernel_name = tf.identity(dense.weights[0])
  bias_name = tf.identity(dense.weights[1])
  logging_weight_hook = tf1.train.LoggingTensorHook(
      tensors=[kernel_name, bias_name],
      every_n_iter=1)
  # Log the training loss by the tensor object.
  logging_loss_hook = tf1.train.LoggingTensorHook(
      {'loss from LoggingTensorHook': loss},
      every_n_secs=3)

  # Pass all hooks to `EstimatorSpec`.
  return tf1.estimator.EstimatorSpec(mode,
                                     loss=loss,
                                     train_op=train_op,
                                     training_hooks=[stop_hook,
                                                     logging_weight_hook,
                                                     logging_loss_hook])

estimator = tf1.estimator.Estimator(model_fn=_model_fn)

# Begin training.
# The training will stop after 2 steps, and the weights/loss will also be logged.
estimator.train(_input_fn)

## TensorFlow 2：使用自定义回调和 Model.fit 记录张量和停止训练

在 TensorFlow 2 中，当您使用内置 Keras `Model.fit`（或 `Model.evaluate`）进行训练/评估时，可以通过定义自定义 Keras `tf.keras.callbacks.Callback` 来配置张量监视和训练停止。随后，将它们传递给 `Model.fit`（或 `Model.evaluate`）的 `callbacks` 参数。（在[编写自己的回调](../..guide/keras/custom_callback.ipynb)指南中了解详情。）

在下面的示例中：

- 要重新创建 `StopAtStepHook` 的功能，请定义一个自定义回调（下称 `StopAtStepCallback`），可以在其中重写 `on_batch_end` 方法以在一定数量的步骤后停止训练。
- 要重新创建 `LoggingTensorHook` 行为，请定义一个自定义回调 (`LoggingTensorCallback`)，可以在其中手动记录和输出记录的张量，因为不支持按名称访问张量。此外，您还可以在自定义回调中实现记录频率。下面的示例将每两步打印一次权重。每 N 秒记录一次之类的其他策略也是可行的。

In [ ]:
class StopAtStepCallback(tf.keras.callbacks.Callback):
  def __init__(self, stop_step=None):
    super().__init__()
    self._stop_step = stop_step

  def on_batch_end(self, batch, logs=None):
    if self.model.optimizer.iterations >= self._stop_step:
      self.model.stop_training = True
      print('\nstop training now')

class LoggingTensorCallback(tf.keras.callbacks.Callback):
  def __init__(self, every_n_iter):
      super().__init__()
      self._every_n_iter = every_n_iter
      self._log_count = every_n_iter

  def on_batch_end(self, batch, logs=None):
    if self._log_count > 0:
      self._log_count -= 1
      print("Logging Tensor Callback: dense/kernel:",
            model.layers[0].weights[0])
      print("Logging Tensor Callback: dense/bias:",
            model.layers[0].weights[1])
      print("Logging Tensor Callback loss:", logs["loss"])
    else:
      self._log_count -= self._every_n_iter

完成后，将新回调（`StopAtStepCallback` 和 `LoggingTensorCallback`）传递给 `Model.fit` 的 `callbacks` 参数：

In [ ]:
dataset = tf.data.Dataset.from_tensor_slices((features, labels)).batch(1)
model = tf.keras.models.Sequential([tf.keras.layers.Dense(1)])
optimizer = tf.keras.optimizers.Adagrad(learning_rate=0.05)
model.compile(optimizer, "mse")

# Begin training.
# The training will stop after 2 steps, and the weights/loss will also be logged.
model.fit(dataset, callbacks=[StopAtStepCallback(stop_step=2),
                              LoggingTensorCallback(every_n_iter=2)])

## 后续步骤

通过以下方式详细了解回调：

- API 文档：`tf.keras.callbacks.Callback`
- 指南：[编写自己的回调](../..guide/keras/custom_callback.ipynb/)
- 指南：[使用内置方法进行训练和评估](https://tensorflow.google.cn/guide/keras/train_and_evaluate)（*使用回调*部分）

此外，您可能还会发现下列与迁移相关的资源十分有用：

- [提前停止迁移指南](early_stopping.ipynb)：`tf.keras.callbacks.EarlyStopping` 是一个内置的提前停止回调
- [TensorBoard 迁移指南](tensorboard.ipynb)：TensorBoard 支持跟踪和显示指标
- [使用辅助逻辑进行训练迁移指南](sessionrunhook_callback.ipynb)：从 TensorFlow 1 中的 `SessionRunHook` 到 TensorFlow 2 中的 Keras 回调